In [25]:
import time
import os


from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException

import pyautogui
from pywinauto import Application
import psutil

In [47]:
instagram_id = os.getenv('INSTAGRAM_ID')
instagram_password = os.getenv('INSTAGRAM_PASSWORD')
driver_path = os.getenv('DRIVER_PATH')
file_path = r"C:\Users\amz_o\Camino.jpg"

In [48]:
def subprocess_for_login():
    cancel_button_korean()
    app_download_alert()
    notification_setting()
    
def cancel_button_korean():
    try:
        button_for_login = driver.find_element_by_xpath("//button[text()='취소']")
        button_for_login.click()
    except NoSuchElementException as e:
        pass

def app_download_alert():
    try:
        button_for_login = driver.find_element_by_xpath("//a[text()='나중에 하기']")
        button_for_login.click()
    except NoSuchElementException as e:
        pass
    
def notification_setting():
    try:
        button_for_login = driver.find_element_by_xpath("//button[text()='나중에 하기']")
        button_for_login.click()
    except NoSuchElementException as e:
        pass

###  로그인 페이지

### 로그인

In [49]:
def input_clear_and_type(input_tag, sentence):
    input_tag.clear()
    input_tag.send_keys(sentence)

### 브라우저설정 

In [50]:
options = Options()
# chrome user agent를 지정해주기
options.add_argument("user-agent=Mozilla/5.0 (iPhone; CPU iPhone OS 11_0 like Mac OS X) AppleWebKit/604.1.38 (KHTML, like Gecko) Version/11.0 Mobile/15A372 Safari/604.1")
driver = webdriver.Chrome(driver_path, options=options)
driver.maximize_window()
driver.implicitly_wait(3)

### 로그인

In [51]:
target_website = "https://www.instagram.com/accounts/login/?hl=ko&source=auth_switcher"
driver.get(target_website)
input_id = driver.find_element_by_xpath("//input[@aria-label='전화번호, 사용자 이름 또는 이메일'][@name='username'][@type='text']")
input_password = driver.find_element_by_xpath("//input[@aria-label='비밀번호'][@name='password'][@type='password']")
button_for_login = driver.find_element_by_xpath("//button[@type='submit'][descendant::div[contains(text(),'로그인')]]")

input_id.clear()
input_id.send_keys(instagram_id)
input_password.clear()
input_password.send_keys(instagram_password)

button_for_login.click()

### 팝업 핸들링

In [52]:
subprocess_for_login()

### 포스팅을 위한 button object 및 click

In [56]:
isNotClicked = True
while isNotClicked:
    try:
        button_for_posting = driver.find_element_by_xpath("//div[@role='menuitem'][@tabindex='0']")
        button_for_posting.click()
        isNotClicked = False
    except:
        subprocess_for_login()

### pyautogui를 이용한 파일다이얼로그 핸들링

In [ ]:
import pyautogui

location_for_input = pyautogui.locateOnScreen('instagram/file_dialog.png', confidence=0.8)
coordinates_for_input = pyautogui.center(location_for_input)
pyautogui.click(coordinates_for_input)
pyautogui.typewrite(file_path) # file 경로를 받아와야함.

location_for_button = pyautogui.locateOnScreen('instagram/open.png', confidence=0.8)
coordinates_for_button = pyautogui.center(location_for_button)
pyautogui.click(coordinates_for_button)

### pywinauto를 이용한 파일다이얼로그 핸들링

In [67]:
import pywinauto

chrome_driver_process_id = driver.service.process.pid # 현재 구동되고있는 크롬드라이버의 process id를 얻는다.
chrome_driver_process = psutil.Process(chrome_driver_process_id)  # 크롬드라이버의 pid로 psutil의 process 객체를 생성한다.
children_process = chrome_driver_process.children(recursive=False)[0] # 크롬드라이버의 자식프로세스인 chrome.exe 프로세스의 id를 얻는다.
children_process_id = children_process.pid
chrome_application = Application(backend='uia').connect(process=children_process_id) # chrome.exe의 pid로 Application 객체를 생성한다.

In [68]:
chrome_application = chrome_application.top_window() # Application객체에서 window specification 객체를 생성한다.
file_dialog = chrome_application.OpenDialog # 크롬에서 file dialog부분만을 얻어온다.
ok_button = file_dialog.child_window(title_re="열기", control_type="Button", auto_id="1") # 경로입력후 열기버튼객체
edit_file_path = file_dialog.child_window(title_re="파일 이름.*", control_type="ComboBox").child_window(title_re="파일 이름.*")
edit_file_path.set_focus() # set focus 메소드는 화면으로 띄워줌 ! (보장됨) AOT
pyautogui.typewrite(file_path) 
ok_button.click()

<uia_controls.ButtonWrapper - '열기(O)', Button, 3713039837550487181>

In [22]:
button_for_click = driver.find_element_by_xpath("//button[text()='다음']") # 파일첨부 후 다음버튼누르기
button_for_click.click() # 클릭하기

### 포스팅하기

In [23]:
text_area = driver.find_element_by_xpath("//textarea[contains(@aria-label,'문구 입력')][contains(@placeholder,'문구 입력')]") # 텍스트입력
text_area.send_keys("인스타그램테스트")
button_for_post = driver.find_element_by_xpath("//button[text()='공유하기']") # 포스팅하기
button_for_post.click()